# Продвинутый Python, семинар 5

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Upd.: Дополнили информацией со всех семинаров :)

## Property

Что? - способ работать с атрибутами

Зачем? - работать с атрибутами, которые не должны быть "публичными" так, чтобы ничего не испортить (проводить валидацию) + не создавать дополнительные названия

Например, создали класс "Дом" с полем "цена". Во время работы с этим полем возникнет потребность изменять этот атрибут. Но цена не может быть чем-то случайным. Цена - это обязательно неотрицательное число. Надо как-то запретить ставить неправильную цену

In [ ]:
class MyClass:
  #можем попытаться хоть как-то ограничить
  a: int = 100

In [ ]:
#К этой переменной есть легкий доступ
v = MyClass()
print(v.a)
#Можно сделать отрицательной
v.a = -123
print(v.a)
#Ууупс, даже строкой можно
v.a = 'Oops'
print(v.a)

100
-123
Oops


Падажжите, тут же даже никак не отсмотрелось, что a должен быть int!  - Если вы помните лекию про typing: структура Python никак не изменилась, "a : int" - это скорее для информации программисту

Таким образом, если мы хотим как-то защитить атрибут, надо что-то написать.

Первая идея: давайте попросим программистов не требовать напрямую v.a, а запрашивать спец-метод v.set\_a (а если нужно вывыести, то get\_a)

In [ ]:

class MyClass:
  a = 100
  def set_a(self, x):

    if type(x) != int:
      print('a must be int')
      return 
    if x < 0:
      print('a must be >= 0')
      return 
    self.a = x
  #def get_a() ...

In [ ]:
v = MyClass()
print(v.a)
v.set_a(4)
print(v.a)
v.set_a(-6664)
print(v.a)
v.set_a('Oops')
print(v.a)

100
4
a must be >= 0
4
a must be int
4


О, теперь работает.

Но есть проблемы: 


*   Если раньше уже был код вида "v.a = 34", то его надо переписывать 
*   Кто-то может нечаянно написать "v.a"
*   Надо помнить, какие атрибуты можно "тыкать" напрямую, какие- нельзя (или для всех атрибутов написать set_name, но это куча лишней копипасты, которую мы не любим)


In [ ]:
#Юзаем декоратор property
class MyClass(object):
  #Пусть настоящий атрибут a станет внутренним
  _a = 100
  #Юзер быдет работать просто с атрибутом a

  #getter
  @property
  def a(self):
    #Пропишем, например, что быдет выдаваться строка, а не просто атрибут
    return 'the a is ' + str(self._a)
  
  #setter
  @a.setter
  def a(self, x):
    if type(x) != int:
      print('a must be int')
    elif x < 0:
      print('a must be >= 0')
    else:  
      self._a = x
  

  

In [ ]:
v = MyClass()
print(v.a)
v.a = 4
print(v.a)
v.a = -6664 
print(v.a)
v.a = 'Oops'
print(v.a)

the a is 100
the a is 4
a must be >= 0
the a is 4
a must be int
the a is 4


Теперь работает и работы с v выглядит привычно и просто

## Контекстный мессенджер

Что это? - то, что задает поведение при использовании конструкции with


Зачем конструкция with?

In [ ]:
#Пример: надо записать что-то в файл
fin = open('file.txt', 'w')
fin.write('some text') #тут может произойти исключение и close не отработает
fin.close() #это можно просто забыть

Важно, что после того, как мы записали все, что нужно, в файл, этот файл надо закрыть. Однако, если произойдет исключение или fin.close() забудем написать, то этого не произойдет.

Вариант: обернуть запись в try except

In [ ]:
#Тут все Ок
fin = open('file.txt', 'w')
try:
    fin.write('some text')
except:
    print('Some error!')
finally:
    fin.close()

In [ ]:
#Тут будет обработана ошибка
fin = open('file.txt', 'w')
try:
    fin.write(some)
except:
    print('Some error!')
finally:
    fin.close()

Some error!


Получается довольно громоздко.

Можно использовать with

In [ ]:
with open('file.txt', 'w') as fin:
     fin.write('some text')

In [ ]:
#Файл закрылся, ошибка вывелась
with open('file1.txt', 'w') as fin:
     fin.write(some)

NameError: ignored

Если работаете с чем-то, что похоже на файл:

*   Присутствует захват/освобождение ресурса
*   Выполняются какие-то действия перед началом работы и в конце

То можно создать свой контекстный менеджер


In [ ]:
#Тут что-то похоже на файл
class MyFile:
    def __init__(self, title):
        self.title = title
        print('MyFile',  self.title, 'opened')
        
    def close(self):
        print('MyFile', self.title, 'closed')
    
    def f(self):
      print('some function of MyFile', self.title)

In [ ]:
#Теперь менеджер
class MyFileManager:
    def __init__(self, title):
        self._file = MyFile(title)

    def __enter__(self):
        return self._file
        
    def __exit__(self, type, value, traceback):
        self._file.close()

И что делает контекстный менеджер?

1. \_\_enter\_\_ должен возвращать объект, который присваивается переменной после as. 

2. \_\_exit\_\_ вызывается при выходе из with (вылетели с ошибкой/закончили работу)

Если ошибка возникает в \_\_init\_\_ или \_\_enter\_\_, тогда блок кода никогда не выполняется и \_\_exit\_\_ не вызывается.


In [ ]:
#Тут мы входим в блок, спокойно работаем внутри и выходим
with MyFileManager('my_file') as r:
  r.f()
  r.f()

MyFile my_file opened
some function of MyFile my_file
some function of MyFile my_file
MyFile my_file closed


In [ ]:
#Теперь добавим ошибку в код (лишний аргумент в функцию + не определили переменную)
with MyFileManager('my_file') as r:
  r.f()
  r.f(some)

MyFile my_file opened
some function of MyFile my_file
MyFile my_file closed


NameError: ignored

Видно, что хоть и произошла ошибка, код прервался, но \_\_exit\_\_ отработал!

Естественно, можно не писать классы, а использовать спец-утилиту

Она назвается contextmanager

Делать ее надо так:


```
from contextlib import contextmanager
@contextmanager
def processor():
    <...> #То, что при __enter__
    yield <...> # Если надо что-то вернуть
    <...> #То, что при __exit__
```



In [ ]:
from contextlib import contextmanager
@contextmanager
def processor(title):

    ## __init__
    fin = MyFile('my_file')
    try: 
      ## __enter__
      yield fin
    finally:
      ##__exit__
      fin.close()
    
with processor('my_file') as file:
    #print(':: processing')
    file.f()
    file.f()


MyFile my_file opened
some function of MyFile my_file
some function of MyFile my_file
MyFile my_file closed


In [ ]:
#Теперь добавим ошибку
with processor('my_file') as file:
    #print(':: processing')
    raise ValueError

MyFile my_file opened
MyFile my_file closed


ValueError: ignored

In [ ]:
#если хотим поработать с обычным файлом
@contextmanager
def open_file(path, mode):
    f = open(path, mode)
    try: 
      yield f
    finally:
      f.close()
with open_file('test.txt', 'w') as file:
    file.write('hello')

In [ ]:
#Добавили ошибку
with open_file('test1.txt', 'w') as file:
    file.write(some)

NameError: ignored

Таким образом, контекстный менеджер - чпочоб работать с "файлами" и конструкцией with

## Статические методы

staticmethod - часто используются для написания "утильных" классов, 
когда главная задача класса - просто группировка методов 
например, мы можем захотеть генерировать себе тестовые данные в отдельно взятом классе,
а потом использовать их в unit-тестах для всего проекта


In [ ]:
from uuid import UUID, uuid4

class Utils:
    @staticmethod
    def capitalize(s: str):
        print(s.capitalize())

class TestDataUtils:
    @staticmethod
    def create_document():
        return {'_id': uuid4(), 'name': 'testName'}

    @staticmethod
    def create_test_worker():
        return {'_id': uuid4(), 'name': 'testName', 'strength': 1, 'skill': 'work'}

    @staticmethod
    def create_test_fighter():
        return {'_id': uuid4(), 'name': 'testName', 'strength': 1, 'skill': 'sword'}

    @staticmethod
    def create_test_archer():
        return {'_id': uuid4(), 'name': 'testName', 'strength': 1, 'skill': 'archery'}

In [ ]:
TestDataUtils.create_test_fighter()

{'_id': UUID('1af79b49-4c99-4a70-bc0f-7d38fc7db2b0'),
 'name': 'testName',
 'strength': 1,
 'skill': 'sword'}

## Продолжаем про ООП 

### Про работу с ООП и в целом

TLDR: сделать все через ООП вам, скорее всего, не понадобится никогда по многим причинам, однако эта практика дает много полезных вещей, которые вам так или иначе потребуются

Есть ли практический смысл все реализовывать через ООП? Очевидно, что нет (иногда это даже вредит, для ознакомления: "Productivity Analysis of Object-Oriented Software Developed in a Commercial Environment")

Однако тогда встает вопрос: а почему изучению ООП вообще посвящают время (в том числе мы тут 3 часа этому посвятили)? Ответов, на самом деле, несколько:

* С помощью ООП можно полностью погрузиться в классы и их концепцию (в том числе в Python)

* Некоторыми вещами вам придется пользоваться (например, контекстным менеджером для подключения к БД)

* ООП дает вам отдельный взгляд на то, как смотреть на написание кода

Мы с вами помним, что есть функции, созданные как способ избежать повторений и сделать код более читабельным (при этом мы немного теряем во времени, ибо запуск функции требует времени).

Так вот, ООП - это возведенный в абсолют концепт читабельности и связей.

Любой сложный проект (или код) требует от вас не только самого написания кода, но и также выстраивание логик-связей-причин (как и любая работа над DWH)

И обычно процесс звучит примерно так:

1. Так, мне надо вот это. А как это можно сделать?..

2. Окей, я понимаю, а что мне надо сделать для этого? (вот часть с логиками)

3. Ну понятно, теперь осталось написать это (желательно эффективно)

### А теперь вернемся к Лолу

Давайте для начала попробуем нарисовать нашу картину того, что мы хотим реализовать

Смотрим на карту:

![](https://gamerall.net/wp-content/uploads/2020/03/oboznachenie-mest-na-lige-legend-s-tochnym-raspolozheniem-kljuchevyh-zon.jpg)

В чем смысл и что тут есть?

1. Есть карта определенных размеров, по которому все ходят

2. Фонтан - место, где оживают персонажи, там же и магазин, где можно купить предметы. Если вражеские игроки заступают на фонтан, то умирают

3. Ингибиторы - место спауна мобой, которые идут по линиям. Если уничтожить ингибитор, то у противника спаунятся супер-миньоны (более сильные)

4. Мобы - 3 ближнего боя, 3 дальнего, спаунятся каждые 30 секунд (раз в три волны еще появляется пушечник). С них падает золото

5. Башни - наносят урон героям, если они атакуют чужих героев, и мобов, если они есть

6. Лесные кэмпы - монстры (с них падает золото и всякие баффы)

7. Сами герои (5 штук с каждой стороны, за которых играют игроки)

Давайте рисовать, как это с ООП сделать!

Будем реализовывать только прототипы (полная реализация будет оставлена в качестве второго дз)

## Dataclass

Зачем? Сделать код более читаемым, понятным, избавиться от копипасты

In [ ]:
#Давайте создадим простой класс
class MyClass:
  a: int
  b: float = 0

In [ ]:
#Можем создать объект этого класса 
v = MyClass()
#Можем тыкать в атрибут
v.b

0

In [ ]:
#А теперь я хочу задать a,b

v = MyClass()
#Ну, выставим вручную после создания - это неудобно
v.a = 12
v.b = 99

v.a, v.b

(12, 99)

In [ ]:
#Просто прописать при создании нельзя - так как нет __init__
#будет ошибка
v = MyClass(5,7)
v.b, v.a

TypeError: ignored

In [ ]:
#Ну давайте напишем __init__
class MyClass:
  def __init__(self, a, b=0):
    self.a = a
    self.b = b

#Теперь можем записывать в скобках при создании
v = MyClass(5)
print(v.b, v.a)

v = MyClass(5,7)
print(v.b, v.a)

0 5
7 5


In [ ]:
#А вот сечас атрибутов ставновится больше

class MyClass:

  def __init__(self, 
               a : int, 
               b : int,
               c : int, 
               d : int, 
               e : int,
               f : int,
               g : int,
               h : int,
               i : int
               ):
    self.a = a
    self.b = b
    self.c = c
    self.d = d
    self.e = e
    self.f = f
    self.g = g
    self.h = h
    self.i = i

v = MyClass(1,2,3,4,5,6,7,8,9)
v.a, v.i

(1, 9)

In [ ]:
#Ладно, теперь я хочу  как-нибудь напечатать иформацию о v
print(v) #это просто выведет класс и расположение 
#Давайте реализуем __repr__ : функцию, возвращающую текстовое представление объекта

In [ ]:

class MyClass:

  def __init__(self, 
               a : int, 
               b : int,
               c : int, 
               d : int, 
               e : int,
               f : int,
               g : int,
               h : int,
               i : int
               ):
    self.a = a
    self.b = b
    self.c = c
    self.d = d
    self.e = e
    self.f = f
    self.g = g
    self.h = h
    self.i = i


  #Больше копипасты Богу копипасты
  def __repr__(self):
    return ('MyClass(' + 'a=' + str(self.a) + ', '+
                        'b=' + str(self.b)+ ', '+
                        'c=' + str(self.c)+ ', '+
                        'd=' + str(self.d)+ ', '+
                        'e=' + str(self.e)+', '+
                        'f=' + str(self.f)+', '+
                        'g=' + str(self.g)+', '+
                        'i=' + str(self.i)+
            ')')

In [ ]:
v = MyClass(1,2,3,4,5,6,7,8,9)
#Хотя бы работает
v

MyClass(a=1, b=2, c=3, d=4, e=5, f=6, g=7, i=9)

Код получился совсем не очень. Уровень копипасты зашкаливает: атрибут повторяется сначала как аргумент в функции, потом как атрибут self., потом, как переменная поданная в функцию. (так еще и функций несколько)

А если аргументов еще больше и названия длинные - то еще некрасивей.

In [ ]:
from dataclasses import dataclass

#Добавляем dataclass и удаляем всю-всю копипасту
@dataclass
class MyCoolClass:
   a : int
   b : int
   c : int
   d : int 
   e : int
   f : int
   g : int
   h : int
   i : int

#Все работает!
u = MyCoolClass(1,2,3,4,5,6,7,8,9)
u

MyCoolClass(a=1, b=2, c=3, d=4, e=5, f=6, g=7, h=8, i=9)

Смотрите, мы избавились от всей глупой копипасты, а все продолжает работать

Таким образом, dataclass сам создал основные методы, тем самым помог избежать посторений, сделал код меньше и более понятным 

Отметим, что мы все еще можем создать собственный \_\_repr\_\_ , \_\_init\_\_ или что-то еще:


In [ ]:
@dataclass
class MyCoolClass1:
   a : int
   #Сделалии стандартный __init__
   def __init__(self, a):
     self.a = a
op = MyCoolClass1(12)
#А __repr__ создан автоматически
op

MyCoolClass1(a=12)

In [ ]:
#Еще пара интересных штук
from dataclasses import astuple, asdict

print('class as python dictionary', asdict(u)) #превращаем в словарь
print('class as python tuple     ', astuple(u)) #превращаем в кортеж

class as python dictionary {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9}
class as python tuple      (1, 2, 3, 4, 5, 6, 7, 8, 9)


In [ ]:
#Со старым MyClass не сработает, так как не использовался декоратор
print('class as python dictionary', asdict(v))
print('class as python tuple     ', astuple(v))

TypeError: ignored

Таким образом, dataclass упрощает работу с классами, делает код понятней, проще + еще пара крутых штук

## Поняли, делаем

Начнем с простого: карта и предметы!

In [ ]:
from dataclasses import dataclass, field
from typing import List
from abc import ABC, abstractmethod

@dataclass
class Item:
    name: str
    unit_price: int
    hp_boost: int
    mana_boost: int
    lethality_boost: int
    armor_boost: int
    magic_armor_boost: int
    speed_boost: int
    depend: List['Item'] = field(default_factory=list) #иногда предмет - это композит других предметов

    def effect(self): #предмет можно использовать!
        pass

In [ ]:
@dataclass
class Map:
    width: int = 500
    height: int = 500

In [ ]:
@dataclass
class LOL_object(ABC):
    hp: int
    current_hp: int
    attack: float = 0
    coords: List[float] = field(default_factory = list)

Чего не хватает? Помните, что на лекции мы говорили, что есть такая штука как регенерация (или наступление ивентов через некоторое время, например, башни перед нексусом имеет свойство восстанаваливать здоровье)

Как же это сделать? Поставить шедулер! Для этого есть библиотека [schedule](https://github.com/dbader/schedule)

In [ ]:
!pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import schedule

In [ ]:
@dataclass
class inhib(LOL_object):
    hp: int
    current_hp: int

    def __post_init__(self):
        super().__init__(self.hp, self.current_hp)

    def regen_hp(self):
        self.current_hp += 20
        print("Health: ", self.current_hp)

    def attack_boost(self):
        self.attack += 1
        print("Attack: ", self.attack)

h = inhib(hp=500, current_hp=500)
schedule.every(10).seconds.do(h.regen_hp)
schedule.every(3).seconds.do(h.attack_boost)

Every 3 seconds do attack_boost() (last run: [never], next run: 2022-10-05 22:16:31)

In [ ]:
while 1:
    schedule.run_pending()

В чем есть проблема? Ну банально в том, чтобы это запустить на фоновом режиме (но как раз об этом мы поговорим при мультипроцессинге), а не сейчас

Сейчас мы все будем делать достаточно банально: создадим на все расписание (в том числе на действие персонажей)

## Развлекаемся с дескрипторами


Дескрипторы можно использовать в качестве атрибутов класса для навешивания на них дополнительной логики. Например, попробуем каждое обращение на чтение и запись сопроводить логированием

In [ ]:
class BadPlaceHolder:
    # owner - класс, в котором объявлен дескриптор
    # name - имя атрибута внутри класса
    # сохраняем имия, чтобы присвоить значение конкретному атрибуту инстанса позже
    def __set_name__(self, owner, name):
        self.public_name = name
        print('set__name__ name= ', name)


    def __set__(self, instance, value):
        print(f'called __set__ with {value}')
        setattr(instance, self.public_name, value)
        pass

    def __get__(self, instance, owner):
        print('called __get__')
        return getattr(instance, self.public_name)


class MyBadClass:
    name1 = BadPlaceHolder()
    surname1 = BadPlaceHolder()

    def __init__(self, name, surname):
        self.name1 = name
        self.surname1 = surname


tmp = MyBadClass('A', 'B')
print(tmp.name1)


тут мы вывалимся с ошибкой, что стек переполнился

давайте попробуем понять и поправить

при обращении к tmp.name1 мы вызвали метод __get__ у атрибута name1, он в свою очередь

будет обращаться к полю name1 нашего объекта tmp через getattr(instance, self.public_name)

как полечить - добавим приватный атрибут!

In [ ]:
class PlaceHolder:

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        print('set__name__ name= ', name)

    def __set__(self, instance, value):
        print(f'called __set__ with {value}')
        setattr(instance, self.private_name, value)
        pass

    def __get__(self, instance, owner):
        print('called __get__')
        return getattr(instance, self.private_name)


class MyClass:
    name1 = PlaceHolder()
    surname1 = PlaceHolder()

    def __init__(self, name, surname):
        self.name1 = name
        self.surname1 = surname


tmp = MyClass('A', 'B')
tmp2 = MyClass('C', 'D')
print(tmp.name1)
print(tmp.surname1)
print(tmp2.name1)
print(tmp2.surname1)

set__name__ name=  name1
set__name__ name=  surname1
called __set__ with A
called __set__ with B
called __set__ with C
called __set__ with D
called __get__
A
called __get__
B
called __get__
C
called __get__
D


## Подчеркивания

Мы использовали функции вида \_\_name\_\_, также очень часто можно встретить _name - почему они пишутся именно так?

Зачем? Во многом это  сделано просто для удобства, читабельности (помогает понять, что за функция/атрибут, где используется).

Использование описано в [гайде по стилю](https://peps.python.org/pep-0008/#descriptive-naming-styles)




```
_name #одно в начале
name_ #одно в конце
__name #два в начале
__name__ #два в начале, два в конце
```



### \_name

\_name используют для "внутренних" переменных - тех, с которыми не предполагается взаимодействие юзера

In [ ]:
import pandas as pd
#Например, если посмотртеь сорс код для pandas.DataFrame, то там есть некоторые атрибуты, начинающиеся с _
df = pd.DataFrame([['A', 10], ['B', 15]])
#Мы можем их достать, но обыно НЕ используем
print(df._mgr)
print(df._constructor_sliced)

BlockManager
Items: RangeIndex(start=0, stop=2, step=1)
Axis 1: RangeIndex(start=0, stop=2, step=1)
NumericBlock: slice(1, 2, 1), 1 x 2, dtype: int64
ObjectBlock: slice(0, 1, 1), 1 x 2, dtype: object
<class 'pandas.core.series.Series'>


К тому же, такие переменные/функции не импортятся (они же "внутренние", не надо их вытаскивать)

In [ ]:
%%writefile func.py

def f():
    return 8
def _g():
    return 0

Writing func.py


In [ ]:
#Теперь из файла импортим всё (а в нем 2 функции: f и _g)
from func import *

#Это импортнулось и работает
print(f())
#Это не испортнулось, выдает ошибку
print(_g())

8


NameError: ignored

### name\_

Это сделано для того, чтобы не путать с какими-то ключевыми словами в Pyhton (+встроенными функциями)

Очень часто затирают функции с простыми именами: 

*   min/max (считают статистику и хотят простое название для переменной)
*   str (делают переменной, где хранят строку)
*   type (хранят какой-то тип)



In [ ]:
#Макс как функция
print(max([1,2,4,2,45,123,7,13,513]))
#max сначала как функция, а потом как переменная с величиной 78
max = max([1,2,3,78])
#просто переменная
print(max)
#Теперь функция уже не работает (так как в макс лежит просто число)
print(max([0,2,5,2,5,4,332]))

513
78


TypeError: ignored

In [ ]:
#Это то же самое, что вызвать число 7
#Точно ошибка
7()

TypeError: ignored

In [ ]:
str = 'qwerty'
#Все, теперь ничего строкой не сделаешь, ошибка
str(567)


TypeError: ignored

Также, есть ключевые слова, такие как class, которые даже не дадут возможности их перетереть

In [ ]:
#class - ключевое слово, выдает ошибку
def func(class):
  return 9

SyntaxError: ignored

In [ ]:
#def тоже --> ошибка
def func(def):
  return 9

SyntaxError: ignored

Таком образом, если очень нужна переменная с названием, похожим на ключевое слово или что-то встроенное, то можно использовать name\_ (или можно попытаться найти осмысленное имя, которые не похоже на ключевое слово)

In [ ]:
def func(class_):
  return class_ + 9
  
func(12)

21

### \_\_name

Такие названия используются, если надо, чтобы в названии атрибута было название класса



In [ ]:
class MyClass:
  __c = 89

In [ ]:
v = MyClass()
#Такое не работает, выдает ошибку
v.__c

AttributeError: ignored

In [ ]:
#А такое ОК
v._MyClass__c

89

In [ ]:
class MySubClass(MyClass):
  __c = 3

In [ ]:
u = MySubClass()
#__c все равно нет в атрибутах
u.__c

AttributeError: ignored

In [ ]:
#А вот и для MyClass, и для MySun
u._MyClass__c, u._MySubClass__c

(89, 3)

### \_\_name\_\_

Это используется для "магии" (штуки для ООП):



```
class MyClass:

  def __inti__(self, ...):
    <...>

  def __repr__(self):
    <...>

  #Для использования <
  def __lt__(self, other):
    <...>
  

  #Для использования float()
  def __float__(self):
    <...>

```

Таких "магических" штук на самом деле много, это примеры нескольких

Не надо придумывать свои \_\_name\_\_ (стандартные использовать лучше так, как задокументировано)

In [ ]:
#Но запретить мне никто не может. Муахаха
class MyClass:
  __c__ = 8

  a = 7
  __d = 9
  
v = MyClass()
v.a, v.__c__

(7, 8)

In [ ]:
#Можем, кст, проверить, какие атрибуты и методы у нас есть для нашего класса
dir(MyClass)

['_MyClass__d',
 '__c__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a']

$⇑$ Видно

*   Выводится довольно сного всего, хотя мы создали только один атрубут. Другие создаются автоматически (как минимум \_\_init\_\_ должен быть, чтобы можно было создать экземпляр объекта)
*   Все, что НЕ было создано нами, имеет название \_\_name\_\_ (то, что создано нами, имеет те имена, что им дали или при использовании \_\_name добавляет название класса)


## Немножко магии с метаклассами

Дока - https://docs.python.org/3/reference/datamodel.html#metaclasses

In [ ]:
class Singleton(type):
    _instances = {}

    # эти методы просто для демонстрации переопределены
    def __new__(cls, clsname, superclasses, attributedict):
        print(f'called new method of {clsname}')
        return type.__new__(cls, clsname, superclasses, attributedict)

    # эти методы просто для демонстрации переопределены
    def __init__(cls, clsname, superclasses, attributedict):
        print(f'called init method {clsname}')
        type.__init__(cls, clsname, superclasses, attributedict)

    # а вот в call происходит самая вкуснятина.
    # call вызывается, когда мы пишем MyClass(), то есть создаем новый
    # экземпляр класса. Метакласс позволяет переопределить создание объекта
    # без каких-либо манипуляций в мета-наследниках
    def __call__(cls, *args, **kwargs):
        print(f"called __call__ method of {cls}")
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class MyClass(metaclass=Singleton):

    def __init__(self):
        print('constructor called')
        

    def __del__(self):
        print('destructor called')
        



f = MyClass()
g = MyClass()
assert f is g


called new method of MyClass
called init method MyClass
called __call__ method of <class '__main__.MyClass'>
constructor called
called __call__ method of <class '__main__.MyClass'>


## Абстрактные методы

Зачем? - сделать так, чтобы при при снаследовании этот метод надо было создавать ОБЯЗАТЕЛЬНО

Зачем? - полезно, если какие-то другие части программы/системы будут предполагать, что этот метод точно существует



In [ ]:
from abc import ABC, abstractmethod
 
class MyClass(ABC):
 
    def __init__(self, value):
        self.value = value
        super().__init__()
    
    #Сделаем do_something абстрактным
    @abstractmethod
    def do_something(self):
        pass

In [ ]:
#Наследуем от MyClass
class MySubClass(MyClass):
  #"Забываем" имплементировать
    pass
#В классе не указали абстрактный метод, который нужен, поэтому получаем ошибку
x = MySubClass(4)

TypeError: ignored

Нельзя наследовать от абстрактного, если не имплементиррованы все абстрактные функции

In [ ]:
class MySubClass1(MyClass):

    def do_something(self):
        return self.value + 1
    
class MySubClass2(MyClass):
   
    def do_something(self):
        return self.value * 2
    
x = MySubClass1(10)
y = MySubClass2(100)

print(x.value, x.do_something())
print(y.value, y.do_something())

10 11
100 200


## Попугай дня

![](https://img2.fonwall.ru/o/lr/birds-couple-macaw-parrot.jpeg?route=mid&amp;h=750)

Ну что же, это ара! (Ара - это не один попугай, это род попугаев)

Самые известные попугаи, которых видно примерно везде (понятно почему - они очень красочные) и проживают в Южной и Центральной Америке

Один из первых попугаев, которых начали содержать в неволе (еще в XI веке иднейцами), их перья использовали в качестве украшений (а еще их ели, вот ужас)

И, естественно, еще одна популярная птица для авикультуры. Однако в чем проблема: они большие! Им нужно очень много пространства (поэтому для них минимум нужна комната, а вообще еще больше пространства) и могут доживать до 90 лет. А еще они орут гроооомко

